# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
#import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(11)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f
8,עד לפני כמה חודשים אני הייתי בטוח שכל חיי אהיה...,m
9,"בן זוגי חגג יום הולדת, ורציתי להפתיע אותו ולסג...",f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

### Name: Lusil Grozdanov




#### Preprocessing Data:
* Remove suffix and prefix
* Remove non-words in Hebrew
* Remove multiply spaces

In [10]:
def CleanData(df):
    
    # Remove non-words in Hebrew
    df['story'] = df['story'].apply(lambda x: re.sub(r'[^\u0590-\u05fe\s]+', '', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'\d+', '', x))
    
    # Remove prefix
    df['story'] = df['story'].apply(lambda x: re.sub(r'\b(כ|כש|וכש|וש|לכש|ו|ב|ש|ה|ל)','', x))
    
    # Remove suffix
    df['story'] = df['story'].apply(lambda x: re.sub(r'(תי|נו|ים)\b', '', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'ות\b', 'ה', x)) 
    
    # Remove multiply spaces
    df['story'] = df['story'].apply(lambda x: re.sub(r'\s+', ' ', x))


In [11]:
# Apply on df_train and df_test

CleanData(df_train)
CleanData(df_test)

In [12]:
df_train['story'][1]

'פני התגייס צבא עשי ל מני מיונ יחידה מובחרה יום סיירה אותו יום אכן קיבל מיון יחידה מאד מאד מיוחדת חיל אני אחד לא אוהב מיוחד מ אמר אתן זה סיכוי אנסה לכת לראה מה יצא מהנסיון מעניין זה גע איזה מקום אזור חיפה אחר מה דקה לבד אמרו ישר הוריד את דבר ישר תחילו הריץ אה ממקום מקום מלמעלה מטה זחילה ריצה כל דרך אפשר דמיין שלב קשי יותר ם יו ריצה מ שחה עה ארוכה הגיע ממקום מקום קבוצה ביחיד אוהל לי יו עוד אנש אחד יה מאד מורעל התקבל יהירדברן רעשן הבן אדם שני יה ן אדם קטנחמד די מופנם הגע אתר א ידע כמה זמן אנח מגיע מיונ מה עוש איך עוש רק ידע אנחש ולכ ביתה תוך די שמועה אמרו המיון אמור היה ין ימ שבוע יו מון אנש התחלה לאט אט תחל הרגיש את דילול אנש מרגע רגע זה יה די קיצוני יי עוש תחרה סמויה ין אוהל ל מי מבין ל אוהל נשאר כי רבה חברי צוה כל אוהל יה לושה אנש יום אחרי יום אנש עפ על ימין על מאל באמת ראי אנח אוהל נשאר הרכב מלא לו תחל דבר על זה אוהל הבחור יהיר תחיל התרברב לצחוק על ולם הבחור שני אמר ו זה א עניין כו בחור יהיר תמיד יה אומר ו מה המפקד א יודע לא ומע זה א משנה עיקר זה הראה הם מה הם רוצ ראה להשמיע הם מה הם

In [13]:
# Using CountVectorizer

def vectorizer(df):
    
    vec = CountVectorizer( min_df=3, max_df=500, max_features=10000 )
    
    X_train = vec.fit_transform(df['story'].tolist())
    df_vectorizer = pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
    df_vectorizer['gender'] = df['gender']
    
    return df_vectorizer

In [14]:
df_vectorizer = vectorizer(df_train)

In [15]:
df_vectorizer

אאוט  אב  אבא  אבד  אבדה  אבה  אבו  אבוא  אבוד  אבוי  אבטחה  אבטלה  אבי  \
0       0   0    0    0     0    0    0     0     0     0      0      0    0   
1       0   0    0    0     0    0    0     0     0     0      0      0    0   
2       0   0    0    0     0    0    0     0     0     0      0      0    0   
3       0   0    8    0     0    0    0     0     0     0      0      0    0   
4       0   0    0    0     0    0    0     0     0     0      0      0    0   
..    ...  ..  ...  ...   ...  ...  ...   ...   ...   ...    ...    ...  ...   
748     0   0    0    0     0    0    0     0     0     0      0      0    0   
749     0   0    0    0     0    0    0     0     0     0      0      0    0   
750     0   0    0    0     0    0    0     0     0     0      0      0    0   
751     0   0    0    0     0    0    0     0     0     0      0      0    0   
752     0   0    0    0     0    0    0     0     0     0      0      0    0   

     אביא  אביב  אביה  אבל  אבן  אבנ  אבק  אג  אגב  אגדה  אגיד  אגיע  אגם  \
0       0     0     0    4    0    0    0   0    0     0     0     0    1   
1       0     0     0    1    0    0    0   0    0     0     0     0    0   
2       0     0     0    5    0    0    0   0    0     0     0     0    0   
3       0     0     0    2    0    0    0   0    0     0     0     0    0   
4       0     0     0    2    0    0    0   0    0     0     0     0    0   
..    ...   ...   ...  ...  ...  ...  ...  ..  ...   ...   ...   ...  ...   
748     0     0     0    0    0    0    0   0    0     0     0     0    0   
749     0     0     0    3    0    0    0   0    0     0     0     0    0   
750     0     0     0    2    0    0    0   0    0     0     0     0    0   
751     0     0     0    0    0    0    0   0    0     0     0     0    0   
752     0     0     0    2    0    0    0   0    0     0     0     0    0   

     אדום  אדומה  אדון  אדיב  אדיר  אדירה  אדם  אדמה  אדע  אדרנלין  אה  אהב  \
0       0      0     0     0     0      0    0     0    0        0   0    0   
1       0      0     0     0     0      0    3     0    0        0   1    0   
2       0      0     0     0     0      0    0     0    0        0   0    0   
3       0      0     0     0     0      0    1     0    0        0   0    0   
4       0      0     0     0     0      0    0     0    0        0   0    0   
..    ...    ...   ...   ...   ...    ...  ...   ...  ...      ...  ..  ...   
748     0      0     0     0     0      0    1     0    0        0   0    0   
749     0      0     0     0     0      0    0     0    0        0   1    0   
750     0      0     0     0     0      0    0     0    0        0   0    0   
751     0      0     0     0     0      0    0     0    0        0   0    0   
752     0      0     0     0     0      0    0     0    0        0   2    0   

     אהבה  אהבו  אהבת  אהוב  אהובה  אהיה  אובדן  אוגוסט  אודה  אודישן  אודם  \
0       0     0     0     0      0     0      0       0     0       0     0   
1       0     0     0     0      0     0      0       0     0       0     0   
2       0     0     0     0      0     0      0       0     0       0     0   
3       0     0     0     0      0     0      0       0     0       0     0   
4       0     0     0     0      0     0      0       0     0       0     0   
..    ...   ...   ...   ...    ...   ...    ...     ...   ...     ...   ...   
748     0     0     0     0      0     0      0       0     0       0     0   
749     0     0     0     0      0     0      0       1     0       0     0   
750     0     0     0     0      0     0      0       0     0       0     0   
751     0     0     0     0      0     0      0       1     0       0     0   
752     0     0     0     0      0     0      0       0     0       0     0   

     אוהב  אוהבת  אוהד  אוהדי  אוהל  אוויר  אווירה  אווירת  אוותר  אוזן  \
0       0      0     0      0     0      0       0       0      0     0   
1       1      0     0      0     7      0       0       0      0     0

In [16]:
#Split the df to femaleDF and maleDF

def split2_female_male(df):
    
    femaleDF = df.replace("f", 1)
    femaleDF = femaleDF.replace("m", 0)
    maleDF = df.replace("m", 1)
    maleDF = maleDF.replace("f", 0)
    
    return femaleDF, maleDF

In [17]:
femaleDF, maleDF = split2_female_male(df_vectorizer)

In [18]:
from sklearn.metrics import make_scorer

def bestParams_bestScore_grid_search(clf, param_grids ,femaleDF, maleDF):
    X_train_fem = femaleDF.drop(columns = ["gender"])
    y_train_fem = femaleDF['gender']
    X_train_M = maleDF.drop(columns = ["gender"])
    y_train_M = maleDF['gender']
    
    grid_search = GridSearchCV(clf, param_grids, cv = 10 , scoring=make_scorer(f1_score, average='macro'), n_jobs = -1)
    clf_female = grid_search.fit(X_train_fem, y_train_fem)
    clf_male = grid_search.fit(X_train_M, y_train_M)
    
    clf_female_score = clf_female.best_score_
    clf_female_params = clf_female.best_params_
    
    clf_male_score = clf_male.best_score_
    clf_male_params = clf_male.best_params_

    clf_score = (clf_female_score + clf_male_score) / 2
    return grid_search.best_params_, clf_score

### The classifiers are:
* LinearSVC()
* MultinomialNB()
* MLPClassifier()
* Perceptron()
* SGDClassifier()
* KNeighborsClassifier

In [19]:
models_bestParams = {}

In [20]:
# LinearSVC()
Linear_svc_parmas = [{'C': [0.01, 0.1, 1, 10, 100], 'penalty': [None, 'l1', 'l2'], 'dual': [False]}]
bestParams,bestScore = bestParams_bestScore_grid_search(LinearSVC(random_state=42), Linear_svc_parmas, femaleDF, maleDF)
models_bestParams['LinearSVC'] = bestParams
print('Best parameters for LinearSVC() : ', bestParams)
print('Best score for LinearSVC() : ', bestScore)

Best parameters for LinearSVC() :  {'C': 100, 'dual': False, 'penalty': 'l1'}
Best score for LinearSVC() :  0.6927026972641543


In [21]:
# MultinomialNB()
MultinomailNB_parmas = [{'alpha': [0.01, 0.1, 0.5, 1], 'fit_prior': [True, False], 'class_prior': [None, [0.5, 0.5], [0.3, 0.7]]}]
bestParams,bestScore = bestParams_bestScore_grid_search(MultinomialNB(), MultinomailNB_parmas, femaleDF, maleDF)
models_bestParams['MultinomialNB'] = bestParams
print('Best parameters for MultinomialNB() : ', bestParams)
print('Best score for MultinomialNB() : ', bestScore)

Best parameters for MultinomialNB() :  {'alpha': 0.5, 'class_prior': [0.3, 0.7], 'fit_prior': True}
Best score for MultinomialNB() :  0.6743960069185501


In [25]:
# MLPClassifier()
MLP_params = [{'hidden_layer_sizes': [(100,100), (50,100), (100,)], 'alpha': [0.0001, 0.001, 0.05], 'learning_rate': ['constant','adaptive']}]
bestParams,bestScore = bestParams_bestScore_grid_search(MLPClassifier(random_state=42),MLP_params,  femaleDF, maleDF)
models_bestParams['MLPClassifier'] = bestParams
print('Best parameters for MLPClassifier() : ', bestParams)
print('Best score for MLPClassifier() : ', bestScore)

Best parameters for MLPClassifier() :  {'alpha': 0.05, 'hidden_layer_sizes': (100, 100), 'learning_rate': 'constant'}
Best score for MLPClassifier() :  0.6571130493562365


In [22]:
# Perceptron()
Perceptron_params = [{'alpha': [0.0001, 0.001, 0.05], 'penalty': [None, 'l2', 'l1', 'elasticnet']}]
bestParams,bestScore = bestParams_bestScore_grid_search(Perceptron(random_state=42), Perceptron_params, femaleDF, maleDF)
models_bestParams['Perceptron'] = bestParams
print('Best parameters for Perceptron() : ', bestParams)
print('Best score for Perceptron() : ', bestScore)

Best parameters for Perceptron() :  {'alpha': 0.001, 'penalty': 'l1'}
Best score for Perceptron() :  0.6731103812825555


In [23]:
# SGDClassifier()
SGD_Params = [{'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
               'penalty': ['l2', 'l1', 'elasticnet'],'alpha': [0.0001, 0.05]}]
bestParams,bestScore = bestParams_bestScore_grid_search(SGDClassifier(random_state=42), SGD_Params, femaleDF, maleDF)
models_bestParams['SGDClassifier'] = bestParams
print('Best parameters for SGDClassifier() : ', bestParams)
print('Best score for SGDClassifier() : ', bestScore)

Best parameters for SGDClassifier() :  {'alpha': 0.0001, 'loss': 'modified_huber', 'penalty': 'l2'}
Best score for SGDClassifier() :  0.6676964915007404


In [24]:
# KNeighborsClassifier()
KNN_Params = [{'n_neighbors': [3, 5, 7]}]
bestParams,bestScore = bestParams_bestScore_grid_search(KNeighborsClassifier(),KNN_Params, femaleDF, maleDF)
models_bestParams['KNeighborsClassifier'] = bestParams
print('Best parameters for KNeighborsClassifier() : ', bestParams)
print('Best score for KNeighborsClassifier() : ', bestScore)

Best parameters for KNeighborsClassifier() :  {'n_neighbors': 3}
Best score for KNeighborsClassifier() :  0.5126173729758821


In [26]:
print(models_bestParams)

{'LinearSVC': {'C': 100, 'dual': False, 'penalty': 'l1'}, 'MultinomialNB': {'alpha': 0.5, 'class_prior': [0.3, 0.7], 'fit_prior': True}, 'Perceptron': {'alpha': 0.001, 'penalty': 'l1'}, 'SGDClassifier': {'alpha': 0.0001, 'loss': 'modified_huber', 'penalty': 'l2'}, 'KNeighborsClassifier': {'n_neighbors': 3}, 'MLPClassifier': {'alpha': 0.05, 'hidden_layer_sizes': (100, 100), 'learning_rate': 'constant'}}


## Predictions
#### (with cross_val_score)

In [27]:
# Calculate the AVG score

def cross_score_avg(clf, femaleDF, maleDF):
    
    Xfemale = femaleDF.drop(columns = ['gender'])
    yfemale = femaleDF['gender']
    Xmale = maleDF.drop(columns = ['gender'])
    ymale = maleDF['gender']

    score_female = cross_val_score(clf, Xfemale, yfemale, scoring = make_scorer(f1_score, average = "macro"),
                                   cv = 10, n_jobs = -1)
    
    score_male = cross_val_score(clf, Xmale, ymale, scoring = make_scorer(f1_score, average = "macro"),
                                 cv = 10, n_jobs = -1)
    
    score_average = (score_female.mean() + score_male.mean() )/2
    
    return score_average

In [28]:
# LinearSVC()

LinearSVC_params = models_bestParams['LinearSVC']
AVG_LinearSVC = cross_score_avg(LinearSVC(**LinearSVC_params), femaleDF, maleDF)
print("Score for LinearSVC: \n", AVG_LinearSVC)

Score for LinearSVC: 
 0.678261961690507


In [29]:
# MultinomialNB()

Multinomial_params = models_bestParams['MultinomialNB']
AVG_Multinomial = cross_score_avg(MultinomialNB(**Multinomial_params), femaleDF, maleDF)
print("Score for MultinomialNB: \n", AVG_Multinomial)

Score for MultinomialNB: 
 0.6708659140127404


In [30]:
# MLPClassifier()

MLP_params = models_bestParams['MLPClassifier']
AVG_MLP = cross_score_avg(MLPClassifier(**MLP_params), femaleDF, maleDF)
print("Score for MLPClassifier: \n", AVG_MLP)

Score for MLPClassifier: 
 0.6560485937895872


In [31]:
# Perceptron()
Perceptron_params =  models_bestParams['Perceptron']
AVG_Perceptron = cross_score_avg(Perceptron(**Perceptron_params),femaleDF, maleDF)
print("Score for Perceptron: \n", AVG_Perceptron)

Score for Perceptron: 
 0.6468882157338445


In [32]:
# SGDClassifier()
SGD_params = models_bestParams['SGDClassifier']
AVG_SGD = cross_score_avg(SGDClassifier(**SGD_params), femaleDF, maleDF)
print("Score for SGDClassifier: \n", AVG_SGD)

Score for SGDClassifier: 
 0.6384452703031138


In [33]:
# KNeighborsClassifier()
KNN_Params = models_bestParams['KNeighborsClassifier']
AVG_KNN = cross_score_avg(KNeighborsClassifier(**KNN_Params),femaleDF, maleDF)
print("Score for KNeighborsClassifier: \n", AVG_KNN)

Score for KNeighborsClassifier: 
 0.5126173729758821


### The best score is for LinearSVC Classifier:
Score for LinearSVC: 0.678261961690507

In [34]:
def best_vec(df_train, df_test):

    df = pd.concat([df_train, df_test], axis=0,ignore_index=True)
    vec = CountVectorizer( min_df=3, max_df=500, max_features=10000 )
    X_train = vec.fit_transform(df["story"])
    
    df_vec = pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
    df_vec['gender'] = df['gender']
    df_vec["test_example_id"] = df["test_example_id"]

    df_train = df_vec.dropna(subset=['gender']).copy()
    df_train = df_train.drop(columns=["test_example_id"])

    df_test =  df_vec.dropna(subset=['test_example_id']).copy()
    df_test = df_test.drop(columns=['gender'])
    
    return df_train, df_test

In [35]:
df_train, df_test = best_vec(df_train, df_test)

In [36]:
femaleDF, maleDF = split2_female_male(df_train)

In [37]:
params = models_bestParams['LinearSVC']
params

{'C': 100, 'dual': False, 'penalty': 'l1'}

In [38]:
params = models_bestParams['LinearSVC']
clf = LinearSVC(C = 100, dual = False, penalty = 'l1')
X = femaleDF.drop(columns = ['gender'])

In [39]:
clf.fit(X, femaleDF['gender'])

LinearSVC(C=100, dual=False, penalty='l1')

In [40]:
# predict for the test_set
y_pred = clf.predict(df_test.drop(columns = ["test_example_id"]))
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,

In [41]:
df_predicted = pd.DataFrame({"test_example_id":df_test["test_example_id"],"predicted_category": y_pred})
df_predicted.reset_index(drop=True, inplace=True)
df_predicted['predicted_category'] = df_predicted['predicted_category'].replace({1: 'f', 0: 'm'})

df_predicted

,test_example_id,predicted_category
0,0.000,m
1,1.000,m
2,2.000,m
3,3.000,m
4,4.000,m
...,...,...
318,318.000,m
319,319.000,m
320,320.000,m
321,321.000,m


In [42]:
df_predicted.head(10)

,test_example_id,predicted_category
0,0.000,m
1,1.000,m
2,2.000,m
3,3.000,m
4,4.000,m
5,5.000,m
6,6.000,m
7,7.000,m
8,8.000,m
9,9.000,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [43]:
df_predicted.to_csv('classification_results.csv',index=False)